In [17]:
import flexs

In [18]:
import editdistance
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pprint
import numpy as np
import json

import flexs
from flexs import baselines
import flexs.utils.sequence_utils as s_utils
import torch.nn.functional as F
import torch

In [19]:
seq_len = 20 # 20 or 50!

In [20]:

from oracles.custom_models.alt_ising_model import AlternatingChainIsingModel

def AltIsingModel(length=50, vocab_size=20):
    return AlternatingChainIsingModel(length=length, vocab_size=vocab_size)


model = AltIsingModel(length=seq_len, vocab_size=20)

In [21]:
from collections import OrderedDict

# enc_len = 50
num_actions = 20


char_pairs = [('A', 0), ('R', 1), ('N', 2), ('D', 3), ('C', 4), ('E', 5), ('Q', 6), ('G', 7), ('H', 8), ('I', 9), ('L', 10), ('K', 11), ('M', 12), ('F', 13), ('P', 14), ('S', 15), ('T', 16), ('W', 17), ('Y', 18), ('V', 19), ('>', 20)]
mol_enc = OrderedDict(char_pairs)
enc_mol = OrderedDict(list(map(lambda x : (x[1], x[0]), char_pairs)))

In [22]:
def seq_to_enc(seq):
    enc = [None for i in range(len(seq))]
    for i in range(len(seq)):
        enc[i] = mol_enc[seq[i]]
    
    return F.one_hot(torch.tensor(enc), num_classes=num_actions).numpy()

In [23]:
def convertor(sequences):
    """
        Does the padding of the sequences to the correct length... w/ the extra chars...
        
        Input: sequences List[str]
        
        Return: list[ndarray]
    """
    
    all_seqs = []
    for seq in sequences:
        all_seqs.append(seq_to_enc(seq)) # Not flattened for this problem
        
    return np.stack(all_seqs)
    
    
    

In [24]:
import pickle

class IsingLandscape(flexs.Landscape):
    """AMP landscape."""

    def __init__(self, seq_len):
        """Create a AMP landscape."""
        super().__init__(name=f"Ising{seq_len}")
        self.alphabet = flexs
        
        self.model = AltIsingModel(length=seq_len, vocab_size=20)


    def _fitness_function(self, sequences):
        """
            Takes as input a list of strings (w/ alphabet of 20)
            
            
            Returns numpy array of scores
        """
        
        np_seqs = convertor(sequences)
        scores = self.model(np_seqs.argmax(-1))
        
        return scores

In [25]:
landscape = IsingLandscape(seq_len)
alph_chars = list(mol_enc.keys())[:-1]
alphabet=''.join(alph_chars)

In [26]:
len(alphabet)

20

In [46]:
query_batch_size = 500
model_queries_per_batch = 500
nRounds = 16

In [47]:
# Start from a random sequence!
rand_seq_len = seq_len
starting_sequence = "".join([np.random.choice(list(alph_chars)) for _ in range(rand_seq_len)])

## Random Explorer

In [33]:



cnn = baselines.models.CNN(len(starting_sequence), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

random_explorer = baselines.explorers.Random(
    cnn,
    rounds=nRounds,
    mu=1,
    starting_sequence=starting_sequence,
    sequences_batch_size=query_batch_size,
    model_queries_per_batch=model_queries_per_batch,
    alphabet=alphabet
)

In [34]:
random_sequences, metadata = random_explorer.run(landscape)
random_sequences

round: 0, top: 1.0, time: 0.000417s
round: 1, top: 3.0, time: 0.949691s
round: 2, top: 3.0, time: 0.858994s
round: 3, top: 4.0, time: 1.656385s
round: 4, top: 4.0, time: 2.113994s
round: 5, top: 4.0, time: 2.916990s
round: 6, top: 4.0, time: 3.538853s
round: 7, top: 5.0, time: 4.218472s
round: 8, top: 5.0, time: 5.265218s
round: 9, top: 5.0, time: 5.743859s
round: 10, top: 5.0, time: 5.936847s
round: 11, top: 5.0, time: 5.984253s
round: 12, top: 5.0, time: 6.929094s
round: 13, top: 5.0, time: 7.721150s
round: 14, top: 5.0, time: 8.436066s
round: 15, top: 5.0, time: 8.507250s
round: 16, top: 5.0, time: 9.341393s


,sequence,model_score,true_score,round,model_cost,measurement_cost
0,VRGNCFKRPAHHICWSMESQ,NaN,1.0,0,0,1
0,VRGNNFKRPAQHICWSMESQ,0.681054,1.0,1,501,501
1,VRGNCFKRPAHHICWSMESH,0.764111,1.0,1,501,501
2,VHGNCFKRPADHIVWSMESQ,0.729104,1.0,1,501,501
3,VPGNCFKRPAHHICWSMWSQ,0.702433,1.0,1,501,501
...,...,...,...,...,...,...
495,TRSLTFSERAHHIMWSWEGQ,2.836648,3.0,16,8016,8001
496,NEGACFMRPPGHICNSMEIQ,1.764232,2.0,16,8016,8001
497,CRGNCFQRPFHHIDPSMESQ,1.946559,2.0,16,8016,8001
498,VEFNCFKFSVFHICMSLEFG,1.368284,1.0,16,8016,8001


## Adalead Explorer

In [43]:
cnn = baselines.models.CNN(len(starting_sequence), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

adalead_explorer = baselines.explorers.Adalead(
    cnn,
    rounds=nRounds,
    starting_sequence=starting_sequence,
    sequences_batch_size=query_batch_size,
    model_queries_per_batch=model_queries_per_batch,
    alphabet=alphabet
)

In [44]:
adalead_sequences, metadata = adalead_explorer.run(landscape)
adalead_sequences

round: 0, top: 1.0, time: 0.000404s
round: 1, top: 3.0, time: 8.307683s
round: 2, top: 5.0, time: 9.624831s


KeyboardInterrupt: 

## Genetic Explorer

In [48]:
cnn = baselines.models.CNN(len(starting_sequence), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

genetic_explorer = baselines.explorers.GeneticAlgorithm(
    cnn,
    
    population_size=8,
    parent_selection_strategy='wright-fisher', # wright-fisher model decides who gets to 'mate'
    beta=0.01,
    children_proportion=0.2,

    rounds=nRounds,
    starting_sequence=starting_sequence,
    sequences_batch_size=query_batch_size,
    model_queries_per_batch=model_queries_per_batch,
    alphabet=alphabet
)

In [49]:
genetic_algo_sequences, metadata = genetic_explorer.run(landscape)

round: 0, top: 2.0, time: 0.000486s
round: 1, top: 4.0, time: 17.190268s
round: 2, top: 5.0, time: 16.474540s
round: 3, top: 7.0, time: 18.348654s
round: 4, top: 8.0, time: 18.183764s


/Users/leofeng/miniconda3/envs/metarlbo_env/lib/python3.7/site-packages/flexs/baselines/explorers/genetic_algorithm.py:93: RuntimeWarning: overflow encountered in exp
  fitnesses = np.exp(scores / self.beta)
/Users/leofeng/miniconda3/envs/metarlbo_env/lib/python3.7/site-packages/flexs/baselines/explorers/genetic_algorithm.py:94: RuntimeWarning: invalid value encountered in true_divide
  probs = torch.Tensor(fitnesses / np.sum(fitnesses))


RuntimeError: invalid multinomial distribution (encountering probability entry < 0)

## CMAES

In [ ]:
cnn = baselines.models.CNN(len(starting_sequence), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

cmaes_explorer = baselines.explorers.CMAES(
    flexs.LandscapeAsModel(landscape),
    
    population_size=10,
    max_iter=200,
    
    rounds=nRounds,
    starting_sequence=starting_sequence,
    sequences_batch_size=query_batch_size,
    model_queries_per_batch=model_queries_per_batch,
    alphabet=alphabet
)

In [ ]:
cmaes_sequences, metadata = cmaes_explorer.run(landscape)

## DynaPPO

In [45]:
dynappo_explorer = baselines.explorers.DynaPPO(  # DynaPPO has its own default ensemble model, so don't use CNN
    landscape=landscape,
    env_batch_size=10,
    num_model_rounds=10,
    rounds=nRounds,
    starting_sequence=starting_sequence,
    sequences_batch_size=query_batch_size,
    model_queries_per_batch=model_queries_per_batch,
    alphabet=alphabet,
)

dynappo_sequences, metadata = dynappo_explorer.run(landscape)
dynappo_sequences

/Users/leofeng/miniconda3/envs/metarlbo_env/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  positive)


BoundedTensorSpec(shape=(), dtype=tf.int64, name='action', minimum=array(0), maximum=array(19))
round: 0, top: 1.0, time: 0.000254s
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=True)` instead.
round: 1, top: 6.0, time: 107.565540s
round: 2, top: 6.0, time: 196.621941s


KeyboardInterrupt: 

## PPO

In [50]:
cnn = baselines.models.CNN(len(starting_sequence), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

ppo_explorer = baselines.explorers.PPO(  # DynaPPO has its own default ensemble model, so don't use CNN
    model=cnn,
    rounds=nRounds,
    starting_sequence=starting_sequence,
    sequences_batch_size=query_batch_size,
    model_queries_per_batch=model_queries_per_batch,
    alphabet=alphabet,
)

ppo_sequences, metadata = ppo_explorer.run(landscape)
ppo_sequences

ValueError: Given `time_step`: TimeStep(
{'discount': array(1., dtype=float32),
 'observation': {'fitness': array([0.2163823], dtype=float32),
                 'sequence': array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 